Dataset link
https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

Model

In [ ]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from keras.callbacks import EarlyStopping
from glob import glob
IMAGESHAPE = [224, 224, 3]
training_data = '/content/train'
testing_data = '/content/test'
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
for each_layer in vgg_model.layers:
 each_layer.trainable = False
classes = glob('/content/train/*')
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()
final_model.compile(
 loss='categorical_crossentropy',
 optimizer='adam',
 metrics=['accuracy']
)
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
 shear_range = 0.2,
 zoom_range = 0.2,
 horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)
training_set = train_datagen.flow_from_directory('/content/train',
 target_size = (224, 224),
 batch_size = 32,
 class_mode = 'categorical')
test_set = testing_datagen.flow_from_directory('/content/test',
 target_size = (224, 224),
 batch_size = 32,
 class_mode = 'categorical')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
fitted_model = final_model.fit(
 training_set,
 validation_data=test_set,
 epochs=30,
 steps_per_epoch=len(training_set),
 validation_steps=len(test_set),
 callbacks=[early_stopping]
)
plot.plot(fitted_model.history['loss'], label='training loss')
plot.plot(fitted_model.history['val_loss'], label='validation loss')
plot.legend()
plot.show()
plot.savefig('LossVal_loss')
plot.plot(fitted_model.history['accuracy'], label='training accuracy')
plot.plot(fitted_model.history['val_accuracy'], label='validation accuracy')
plot.legend()
plot.show()
plot.savefig('AccVal_acc')
final_model.save('Mini.h5')

Implementation

In [ ]:
import gradio as gr
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model = load_model('Mini.h5')
def preprocess_image(img):
 img = img.resize((224, 224))
 img_array = image.img_to_array(img)
 img_array = np.expand_dims(img_array, axis=0)
 img_data = preprocess_input(img_array)
 return img_data
def predict_pneumonia(img):
 img_data = preprocess_image(img)
 pneumonia_prediction = model.predict(img_data)
 if pneumonia_prediction[0][0] > pneumonia_prediction[0][1]:
 result = "Person is safe."
 else:
 result = "Person is affected with Pneumonia."
 return result
iface = gr.Interface(
 fn=predict_pneumonia,
 inputs=gr.Image(type='pil', label="Upload a chest X-ray image"),
 outputs=gr.Textbox(type='text',label='Result')
)
iface.launch()